In [ ]:
import qiskit
from qiskit_experiments.framework import ParallelExperiment
from qiskit_experiments.library import StateTomography
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister

from qiskit.visualization import plot_state_city

from qiskit_aer import AerSimulator
from qiskit.providers.fake_provider import FakePerth

from math import pi

backend = AerSimulator.from_backend(FakePerth())

In [ ]:
# state preparation circuit
# num_qubits = 4
# qc = QuantumCircuit(4)  
# qc.h(0)
# qc.s(0)
# for i in range(1, num_qubits):
#     qc.cx(0, i)

qr = QuantumRegister(3, 'q')
anc = QuantumRegister(1, 'anc')
cr = ClassicalRegister(3, 'c')
qc = QuantumCircuit(qr, anc, cr)

qc.x(anc[0])
qc.h(anc[0])
qc.h(qr[0:3])
qc.cx(qr[0:3], anc[0])
qc.h(qr[0:3])
qc.barrier(qr)
# qc.measure(qr, cr)

# QST experiment
qstexp1 = StateTomography(qc)
qstdata1 = qstexp1.run(backend, seed_simulation=1).block_for_results()

# Print results
for result in qstdata1.analysis_results():
    print(result)

# print(qstdata1.analysis_results('state').value)
# print(qstdata1.analysis_results('state').device_components)
# print("state_fidelity = {:.5f}".format(qstdata1.analysis_results('state_fidelity').value))

# 5-qubit bell state
# q5 = QuantumRegister(5)
# bell5 = QuantumCircuit(q5)
# bell5.h(q5[0])
# for j in range(4):
#     bell5.cx(q5[j], q5[j + 1])
qc.draw('mpl')

In [ ]:

# plot_bloch_multivector(qstdata1.analysis_results('state').value, title='Density Matrix')
plot_state_city(qstdata1.analysis_results('state').value, title='Density Matrix')
# plot_state_hinton(qstdata1.analysis_results('state').value, title='Density Matrix')
# plot_state_paulivec(qstdata1.analysis_results('state').value, title='Density Matrix')
# plot_state_qsphere(qstdata1.analysis_results('state').value)

In [ ]:
num_qubits = 5
gates = [qiskit.circuit.library.RXGate(i * pi / (num_qubits - 1)) for i in range(num_qubits)]
subexps = [StateTomography(gate, physical_qubits=[i]) for i, gate in enumerate(gates)]
parexp = ParallelExperiment(subexps)
pardata = parexp.run(backend, seed_simulation=1).block_for_results()
for i, expdata in enumerate(pardata.child_data()):
    
    print(f'\nParallel exp {i}')
    print('state: {}'.format(expdata.analysis_results('state').value))
    print('device_components: {}'.format(expdata.analysis_results('state').device_components))
    print('state_fidelity: {:.5f}'.format(expdata.analysis_results('state_fidelity').value))
    